# Chap03 - Word2Vec

## 3.1 추론 기반 기법과 신경망

### 3.1.1 통계 기반 기법의 문제점

통계 기반 기법에서는 주변 단어의 빈도를 기반으로 단어를 표현했다. 단어의 Co-occurrence Matrix를 만들고, 그 행렬에 SVD를 적용하여 밀집벡터(단어의 분산표현)을 구했다. 하지만 이런 방식은 대규모 말뭉치(corpus)를 다룰 때 문제가 발생한다. 

통계 기반 기법은 말뭉치 전체의 통계(동시발생 행렬과 PPMI 등)를 이용해 **단 1회의 처리**(SVD 등)만에 단어의 분산 표현을 얻는다. 하지만 신경망을 이용한 **추론 기법**에서는 미니 배치(학습 데이터의 일부)를 이용해 학습한다.

이처럼 신경망은 미니 배치(mini-batch)로 학습하기 때문에 대규모의 말뭉치에서도 학습시킬 수 있다.

### 3.1.2 추론 기반 기법 개요

추론 기법에서의 추론이란 주변 단어(맥락, context)가 주어졌을 때 "?"에 어떤 단어가 들어가는지 추측하는 것을 말한다.

![](./images/inference.png)

이러한 추론 문제를 방복해서 풀면서 단어의 출현 패턴을 학습한다. 모델은 맥락(context) 정보를 입력받아 각 단어의 출현 확률을 출력한다. '모델 관점'에서 보면 다음과 같다.

![](./images/inference02.png)



### 3.1.3 신경망에서의 단어 처리

신경망의 입력층에서의 뉴런 수를 **고정**하기 위해 단어를 일정 길이를 갖는 **고정 길이 벡터**로 변환 해준다. 이때 사용하는 대표적인 방법이 **원핫** 벡터(one-hot vector)다.

| 단어(텍스트) | 단어 ID | one-hot vector        |
| ------------ | ------- | --------------------- |
| you          | 0       | [1, 0, 0, 0, 0, 0, 0] |
| goodbye      | 2       | [0, 0, 1, 0, 0, 0, 0] |


<img src="./images/one-hot.png" height="65%" width="65%" />

위와 같이 원-핫 벡터로 나타낸 단어 벡터를 신경망의 입력으로 주입하고 완전연결계층(FC layer)를 통해 원-핫 벡터로 표현된 단어를 밀집 벡터로 변환해줄 수 있다.

![](./images/nn.png)

In [1]:
import numpy as np

c = np.array([[1, 0, 0, 0, 0, 0, 0]])  # 입력 (one-hot)
W = np.random.randn(7, 3)  # 가중치 (랜덤한 값), Word vector
h = np.matmul(c, W)  # 은닉층 노드
print(h)

[[ 1.08641249 -0.82780485 -0.29980432]]


위의 코드에서 `c`는 원-핫 표현이며 단어 ID에 대응하는 원소만 1이고 그 외에는 0인 벡터다. 따라서, `c`와 `W`의 행렬곱은 가중치(`W`)의 행벡터 하나를 **뽑아낸** 것과 같다.

![](./images/matmul.png)

In [4]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul

c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7, 3)
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[-0.38687194  0.82963359  0.87623966]]


## 3.2 단순한 Word2Vec

### 3.2.1 CBOW 모델의 추론 처리

**CBOW**(Continuous Bag-Of-Words) 모델은 맥락(context, 주변 단어)로부터 타깃(target, 중심 단어)을 추측하는 신경망이다. 

<img src="./images/cbow.png" height="65%" width="65%" />

- 모델의 입력은 맥락(context) 즉, 주변 단어다.

- 은닉층의 뉴런은 입력층의 완전연결계층에 의해 변환된 값이 되는데, CBOW에서는 `wiindow_size * 2`($C$) 개수 만큼 입력이 되므로 전체의 **평균**(average)을 해준다.  


$$
\mathbf{h} = \frac{1}{C} (h_1 + \cdots h_c)
$$


- 출력층의 뉴런은 해당 단어의 개수(`vocab_size`)와 같고, 뉴런 하나하나가 각 단어에 대응한다. 출력층 뉴런은 각 단어의 **'점수'**(score, softmax를 지나기전 상태)를 뜻하며, 값이 높을 수록 대응 단어의 출현 확률도 높아진다.

- 위의 그림에서 $\mathbf{W}_{\text{in}}$의 각 행(row)과 $\mathbf{W}_{\text{out}}$의 각 열(column)이 바로 단어의 분산 표현이 된다. $\mathbf{W}_{\text{in}}$와 $\mathbf{W}_{\text{out}}$ 중 어느것을 써도 상관 없지만 보통 $\mathbf{W}_{\text{in}}$을 사용하는 듯 하다.

<img src="./images/word_vec.png" height="65%" width="65%" />

In [8]:
# CBOW모델의 추론 처리 구현
# chap03/cbow_predict.py
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul

# 샘플 맥락 데이터
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

# 가중치 초기화
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

# 계층 생성
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

# 순전파
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)  # average
s = out_layer.forward(h)  # score

print(s)

[[-0.57328213 -0.16244313 -0.67467078  0.64884723  0.02103023 -0.62611472
  -1.48166945]]


### 3.2.2 CBOW 모델의 학습

CBOW 모델의 학습에서는 올바른 예측을 할 수 있도록 가중치를 조정한다. 그 결과 가중치 $\mathbf{W}_{\text{in}}$와 $\mathbf{W}_{\text{out}}$에 각 단어의 벡터가 학습된다.

Word2Vec 모델은 단어 출현 패턴을 학습할 때 사용한 말뭉치(corpus)로부터 학습한다. 그렇기 때문에 말뭉치가 다르면 학습 후 얻게 되는 단어의 분산 표현도 달라진다.

<img src="./images/cbow02.png" height="65%" width="65%" />

### 3.2.3 Word2Vec의 가중치와 분산 표현

Word2Vec 모델에서는 $\mathbf{W}_{\text{in}}$의 각 행(row)과 $\mathbf{W}_{\text{out}}$의 각 열(column)이 바로 단어의 분산 표현이 된다. 

최종적으로 이용하는 단어의 분산 표현은 다음과 같이 세 가지 선택 사안이 있다.

- 입력 측의 가중치($\mathbf{W}_{\text{in}}$)만 이용한다.

- 출력 측의 가중치($\mathbf{W}_{\text{out}}$)만 이용한다.

- 양쪽 가중치를 모두 이용한다. ($\mathbf{W}_{\text{in}}$와 $\mathbf{W}_{\text{out}}$의 합 등)


Word2Vec에서 특히 skip-gram 모델에서는 '입력 측의 가중치($\mathbf{W}_{\text{in}}$)만 이용한다'가 가장 대중적인 선택이다. 

<img src="./images/word_vec.png" height="65%" width="65%" />